## Deliverable 2. Create a Customer Travel Destinations Map.

In [29]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [36]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv", index_col=0)
city_data_df.head()

,City,Country,Lat,Lng,Max Temperature,Humidity,Cloudiness,Wind Speed,Current Description
0,Busselton,AU,-33.6500,115.3333,71.10,66,10000,17.76,clear sky
1,Saint Paul Harbor,US,57.7900,-152.4072,30.06,89,4760,10.18,overcast clouds
2,Carnarvon,AU,-24.8667,113.6333,77.07,83,10000,5.75,clear sky
3,Vaini,TO,-21.2000,-175.2000,82.18,80,10000,20.13,overcast clouds
4,Saldanha,ZA,-33.0117,17.9442,68.16,70,10000,24.02,clear sky


In [37]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input('Enter the minimum desirable temperature: '))
max_temp = float(input('Enter the maximum desirable temperature: '))

Enter the minimum desirable temperature: 75
Enter the maximum desirable temperature: 85


In [38]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
city_filtered_df = city_data_df.loc[(city_data_df['Max Temperature'] >= min_temp) &
                                    (city_data_df['Max Temperature'] <= max_temp)]
city_filtered_df.tail()

,City,Country,Lat,Lng,Max Temperature,Humidity,Cloudiness,Wind Speed,Current Description
694,Atar,MR,20.5169,-13.0499,79.90,17,10000,3.76,broken clouds
695,Kantunil,MX,20.8000,-89.0333,75.92,87,10000,3.85,scattered clouds
703,Bintulu,MY,3.1667,113.0333,77.41,100,10000,0.00,scattered clouds
704,Trairi,BR,-3.2778,-39.2689,79.14,80,10000,5.93,light rain
708,Kaputa,ZM,-8.4689,29.6619,78.76,67,10000,4.52,light rain


In [39]:
# 4a. Determine if there are any empty rows.
city_filtered_df.isna().sum()

City                   0
Country                2
Lat                    0
Lng                    0
Max Temperature        0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [40]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = city_filtered_df.dropna()
clean_df.isna().count()

City                   171
Country                171
Lat                    171
Lng                    171
Max Temperature        171
Humidity               171
Cloudiness             171
Wind Speed             171
Current Description    171
dtype: int64

In [68]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temperature", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.tail(5)

,City,Country,Max Temperature,Current Description,Lat,Lng,Hotel Name
694,Atar,MR,79.90,broken clouds,20.5169,-13.0499,
695,Kantunil,MX,75.92,scattered clouds,20.8000,-89.0333,
703,Bintulu,MY,77.41,scattered clouds,3.1667,113.0333,
704,Trairi,BR,79.14,light rain,-3.2778,-39.2689,
708,Kaputa,ZM,78.76,light rain,-8.4689,29.6619,


In [ ]:
# TESTING
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key}

a = 43.6187102
b = -116.2146068

params['location'] = f'{a}, {b}'
print(params['location'])
response = requests.get(url, params=params).json()

In [ ]:
hotel_df['Hotel Name']

In [142]:
# TESTING
hotel_df_2 = hotel_df

hotel_df_2.loc[2, 'Hotel Name'] = 'El Hotelon'
hotel_df_2



,City,Country,Max Temperature,Current Description,Lat,Lng,Hotel Name
2,Carnarvon,AU,77.07,clear sky,-24.8667,113.6333,El Hotelon
3,Vaini,TO,82.18,overcast clouds,-21.2000,-175.2000,
5,Bengkulu,ID,79.43,broken clouds,-3.8004,102.2655,
12,Rikitea,PF,77.52,overcast clouds,-23.1203,-134.9692,
17,Cabedelo,BR,84.87,broken clouds,-6.9811,-34.8339,
...,...,...,...,...,...,...,...
694,Atar,MR,79.90,broken clouds,20.5169,-13.0499,
695,Kantunil,MX,75.92,scattered clouds,20.8000,-89.0333,
703,Bintulu,MY,77.41,scattered clouds,3.1667,113.0333,
704,Trairi,BR,79.14,light rain,-3.2778,-39.2689,


In [ ]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame   
    lat = row['Lat']
    lng = row['Lng']
    params['location'] = f'{lat}, {lng}'
    
    # 6d. Set up the base URL for the Google Directions (PLACES???) API to get JSON data
    url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

    # 6e. Make request and retrieve the JSON data from the search.
    response = requests.get(url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        first_hotel = response['results'][0]['name']
        hotel_df.loc[index, 'Hotel Name'] = first_hotel
    except:
        print(f'Hotel not found!!!')
        

Hotel not found!!!
Hotel not found!!!
Hotel not found!!!
Hotel not found!!!
Hotel not found!!!
Hotel not found!!!
Hotel not found!!!
Hotel not found!!!
Hotel not found!!!
Hotel not found!!!
Hotel not found!!!


In [ ]:
# TESTING

# Google Directions
https://maps.googleapis.com/maps/api/directions/json?origin=Disneyland&destination=Universal+Studios+Hollywood&key=YOUR_API_KEY

# Google Places
https://maps.googleapis.com/maps/api/place/nearbysearch/output?parameters    

In [75]:
# 7. Drop the rows where there is no Hotel Name.


In [10]:
# 8a. Create the output File (CSV)

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 

# 11b. Display the figure
